In [1]:


!pip install -q rouge_score sacrebleu bert_score

import json
import os
from rouge_score import rouge_scorer
import sacrebleu
from bert_score import score as bert_score

# === Step 1: Define JSON paths ===
file_paths = {
    "GPT": "/content/drive/MyDrive/history_eval_set2.json",
    "Gemini": "/content/drive/MyDrive/h1.json",
    "DeepSeek": "/content/drive/MyDrive/history_eval_set3.json",
}

# === Step 2: Evaluation Function ===
def evaluate_model(json_path):
    """Evaluate one model output file and return averaged ROUGE, BLEU, and BERTScore."""

    if not os.path.exists(json_path):
        raise FileNotFoundError(f"File not found: {json_path}")

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    if not isinstance(data, list) or len(data) == 0:
        raise ValueError(f"{json_path} must contain a list of examples.")

    # --- Auto-detect keys ---
    sample_keys = list(data[0].keys())
    ref_key = next(
        (k for k in sample_keys if "actual" in k.lower() or "reference" in k.lower() or ("answer" in k.lower() and "gen" not in k.lower())),
        None,
    )
    gen_key = next(
        (k for k in sample_keys if "gen" in k.lower() or "pred" in k.lower() or "model" in k.lower()),
        None,
    )

    if not ref_key or not gen_key:
        raise KeyError(f"Could not detect reference/generated keys in {json_path}. Found keys: {sample_keys}")

    # --- Initialize metrics ---
    results = {"ROUGE": [], "BLEU": [], "BERTScore": []}
    scorer = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)

    # --- Compute metrics for each example ---
    for item in data:
        ref = str(item[ref_key]).strip()
        hyp = str(item[gen_key]).strip()
        if not ref or not hyp:
            continue

        # ROUGE
        scores = scorer.score(ref, hyp)
        rouge_avg = (scores["rouge1"].fmeasure + scores["rougeL"].fmeasure) / 2
        results["ROUGE"].append(rouge_avg)

        # BLEU
        bleu = sacrebleu.corpus_bleu([hyp], [[ref]]).score
        results["BLEU"].append(bleu)

        # BERTScore
        P, R, F1 = bert_score([hyp], [ref], lang="en", verbose=False)
        results["BERTScore"].append(float(F1[0]))

    # --- Compute averages ---
    summary = {
        "ROUGE": sum(results["ROUGE"]) / len(results["ROUGE"]) if results["ROUGE"] else 0,
        "BLEU": sum(results["BLEU"]) / len(results["BLEU"]) if results["BLEU"] else 0,
        "BERTScore": sum(results["BERTScore"]) / len(results["BERTScore"]) if results["BERTScore"] else 0,
    }

    return summary


# === Step 3: Evaluate all models ===
print("⚙️ Evaluating models... (BERTScore may take a few minutes)\n")

all_results = {}
for model_name, path in file_paths.items():
    print(f"➡️ Evaluating {model_name}...")
    all_results[model_name] = evaluate_model(path)

# === Step 4: Display results ===
print("\n===============================")
print("📊 MULTI-MODEL EVALUATION SUMMARY")
print("===============================")

for model, metrics in all_results.items():
    print(f"\n🧠 Model: {model}")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.3f}")

print("\n===============================")
print("✅ All model evaluations complete!")


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.8 MB/s eta 0:00:00
⚙️ Evaluating models... (BERTScore may take a few minutes)

➡️ Evaluating GPT...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

➡️ Evaluating Gemini...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

➡️ Evaluating DeepSeek...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho


📊 MULTI-MODEL EVALUATION SUMMARY

🧠 Model: GPT
  ROUGE: 0.729
  BLEU: 23.445
  BERTScore: 0.926

🧠 Model: Gemini
  ROUGE: 0.751
  BLEU: 20.263
  BERTScore: 0.931

🧠 Model: DeepSeek
  ROUGE: 0.709
  BLEU: 19.748
  BERTScore: 0.922

✅ All model evaluations complete!
